Привет! меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за  качественную работу, мне она понравилась!
- Я оставил некоторые советы надеюсь они будут полезными или интересными.
- Техника бутстрап применилась корректно, было проведено тщательное исследование и выбран правильный регион!
- Отправляю проект назад, чтобы у тебя была возможность задать вопросы, если они у тебя есть. Если их нет, то можешь просто отправить проект еще раз и я его зачту



# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

###  Импортируем необходимые библиотеки и загружаем датасеты

In [1]:
import math
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
from tqdm.notebook import tqdm
from scipy import stats as st
import matplotlib.pyplot as plt
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas_profiling'

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>


In [ ]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

### Посмотрим 3 датасет

In [ ]:
data_0.head(20)

In [ ]:
data_0.info()

In [ ]:
data_0.describe()

### Посмотрим 2 датасет

In [ ]:
data_1.head(20)

In [ ]:
data_1.info()

In [ ]:
data_1.describe()

### Посмотрим 3 датасет

In [ ]:
data_2.head(20)

In [ ]:
data_2.info()

In [ ]:
data_2.describe()

### Построим графики зависимостей значений в таблице

In [ ]:
plt.title('Графики распределения запасов сырья в регионах')
ax=data_0['product'].plot(kind='hist',figsize=(12,7),grid=True, legend=True, alpha=0.7, histtype='step', linewidth=5, label='1 регион')
ax1=data_1['product'].plot(kind='hist',figsize=(12,7),grid=True, legend=True, alpha=0.7,histtype='step', linewidth=5, label='2 регион', ax=ax)
data_2['product'].plot(kind='hist',figsize=(12,7),grid=True, legend=True, alpha=0.7,histtype='step', linewidth=5, label='3 регион', ax=ax1)
plt.xlabel('Распределение данных по запасам сырья в регионах')
plt.ylabel('Запасы сырья, тыс.баррелей')
plt.show()

### Ещё раз посмотрим на общую информацию о каждом датасете

In [ ]:
data_0.profile_report()

In [ ]:
data_1.profile_report()

In [ ]:
data_2.profile_report()

**Вывод** <br>
Данные успешно загружены и не требуют дополнительной предобработки. На графике отчётливо показана разительно отличающаяся кривая распределения запасов сырья по скважинам 2 региона. Так же заметна довольно высокая корреляция между признаком f2 и целевым признаком product.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные осмотрены, но можно было бы еще как минимум изучить корреляции, так как мы планируем использовать линейные модели, а для них это важно. 
</div>


<div class="alert alert-block alert-info">
<b>Совет:</b>  Советую обратить внимание на библиотеки sweetviz и pandas_profiling помогут в проведении более тщательного EDA анализа. Исследовательский анализ можно делать и с помощью ручного вызова функций дефолтных библиотек. Данные библиотеки хороши для максимизации комфорта презентации результатов анализа бизнес-пользователям.  

Очень просты в использоовании, на вход кладется датафрейм: pandas_profiling.ProfileReport(df) 
</div>


<div class="alert alert-block alert-warning">
<b>Изменения:</b> Не стал менять прошлый вариант, но добавил ещё один раздел в котором использовал pandas_profiling. Так же немного изменил вывод (добавил про корреляцию)
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Советую прфоайлеров во время обучения не злоупотреблять, чтобы руки не забыл как делать EDA анализ)
</div>

## Обучение и проверка модели

### Разделин данные на обучающую и валидационную выборки в соотношении 75:25

In [ ]:
data_0_train, data_0_valid = train_test_split(data_0, test_size=0.25, random_state=12345)
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)

In [ ]:
features_train_0 = data_0.drop(['product', 'id'], axis=1)
target_train_0 = data_0['product']

features_valid_0 = data_0.drop(['product', 'id'], axis=1)
target_valid_0 = data_0['product']

In [ ]:
features_train_1 = data_1.drop(['product', 'id'], axis=1)
target_train_1 = data_1['product']

features_valid_1 = data_1.drop(['product', 'id'], axis=1)
target_valid_1 = data_1['product']

In [ ]:
features_train_2 = data_2.drop(['product', 'id'], axis=1)
target_train_2 = data_2['product']

features_valid_2 = data_2.drop(['product', 'id'], axis=1)
target_valid_2 = data_2['product']

### Обучим модель и сделаем предсказания на валидационной выборке

In [ ]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
prediction_0 = model_0.predict(features_valid_0)

In [ ]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
prediction_1 = model_1.predict(features_valid_1)

In [ ]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
prediction_2 = model_2.predict(features_valid_2)

### Найдём средний запас предсказанного сырья и RMSE модели

In [ ]:
comparison_tabl = pd.DataFrame(index=['Cредний запас предсказанного сырья','RMSE'], columns=['1 регион','2 регион','3 регион'])
comparison_tabl['1 регион'] = prediction_0.mean(), mean_squared_error(target_valid_0, prediction_0)**0.5
comparison_tabl['2 регион'] = prediction_1.mean(), mean_squared_error(target_valid_1, prediction_1)**0.5
comparison_tabl['3 регион'] = prediction_2.mean(), mean_squared_error(target_valid_2, prediction_2)**0.5
comparison_tabl

**Вывод** <br>
Модели обучены и проверены на качество предсказания метрикой RMSE. Исходя из полученных данный в нашей сводной таблицы, можно заметить, что величина RMSE во втором регионе сильно отличается от двух других, это связано с тем, что модель обучилась на практически равных по величине данных, и величина ошибки этой модели не может сильно варьирываться.

<div class="alert alert-block alert-success">
<b>Успех:</b> Шаг проделан хорошо, все метрики посчитаны корректно
</div> 


<div class="alert alert-block alert-info">
<b>Совет </b>  Можно было бы использовать циклы/функции и избежать дублирования кода.
</div>

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных

При разведке региона исследуюем 500 точек, из которых с помощью машинного обучения выберем 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс.
рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выберем регион с наибольшей средней прибылью

In [ ]:
# Количество скважин
COUNT_OF_WELLS = 200

# Бюджет
BUDGET = 10_000_000_000

# Цена одной скважины
PRICE_ONE_WELL = BUDGET / COUNT_OF_WELLS
#50000000

# Доход с одной единицы продукта
PRICE_ONE_THOUSAND_BARRELS = 450_000

<div class="alert alert-block alert-info">
    
<b>Совет:</b> Константы принято записывать в ВЕРХНЕМ РЕГИСТРЕ =) Читем здесь https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29
    

</div> 

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Изменил регистр
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Есть
</div>

In [ ]:
product_mean_0 = data_0['product'].mean()
product_mean_1 = data_1['product'].mean()
product_mean_2 = data_2['product'].mean()

print('Cредний запас сырья в 1 регионе: %.2f млн.баррелей'%product_mean_0)
print('Cредний запас сырья в 2 регионе: %.2f млн.баррелей'%product_mean_1)
print('Cредний запас сырья в 3 регионе: %.2f млн.баррелей'%product_mean_2)

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [ ]:
v_prod = PRICE_ONE_WELL / PRICE_ONE_THOUSAND_BARRELS
print('Объём каждой скважины для безубыточной разработки должен составлять не менее: %.2f'%v_prod,'тыс.баррелей')

**Вывод** <br>
Исходя из полученных данных видно, что среднее количество запасов сырья по регионам недостаточное даже для окупаемости разработки, поэтому требуется выбрать скважины с максимальными запасами.

<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен  
</div>

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

### Напишем функцию 

In [ ]:
def profit(prediction, target):
    probs_sorted = pd.Series(prediction, index=target.index).sort_values(ascending=False)
    max_pred = probs_sorted[:200].sum()
    profit_pred=(max_pred * PRICE_ONE_THOUSAND_BARRELS) - BUDGET
    print('Суммарный запас сырья в 200 лучших скважинах составит: %.2f тыс.баррелей'%max_pred)
    print('Прибыль для полученного объёма сырья составит %.2f млн.рублей'%(profit_pred/1000000))

<div class="alert alert-block alert-success">
<b>Успех:</b> Ф-ция для подсчета прибыли определена верно
</div>


### Прибыль для первого региона

In [ ]:
profit(prediction_0, target_valid_0)

### Прибыль для второго региона

In [ ]:
profit(prediction_1, target_valid_1)

### Прибыль для третьего региона

In [ ]:
profit(prediction_2, target_valid_2)

**Вывод** <br>
Отлично, все три региона потенциально прибыльные. Осталось выяснить, какой регион наиболее выгодный с точки зрения окупаемости вложений и рисков убытка.

## Посчитаем риски и прибыль для каждого региона

In [ ]:
def profit_1(prediction, target, count):
    probs_sorted = pd.Series(prediction).sort_values(ascending=False)
    selected_points = target[probs_sorted.index][:count]
    product = selected_points.sum()
    revenue = product * PRICE_ONE_THOUSAND_BARRELS
    return revenue - BUDGET

<div class="alert alert-block alert-info">
<b>Совет:</b> З
</div>


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Тут сове
</div>

<div class="alert alert-block alert-info">
<b>Совет[2]:</b> Почему-то комментарии затерлись. Я хотел написать, зачем дублировать функции прибыли?)
</div>


In [ ]:
def bootstap(target, prediction):
    values = []
    count=0
    state = np.random.RandomState(12345)
    predict_e = pd.Series(prediction, index=target.index)
    
    for i in tqdm(range(1000)):
        target_subsample = target.sample(n=500,replace=True, random_state=state)
  
        probs_subsample = predict_e[target_subsample.index]
        values.append(profit_1(probs_subsample, target_subsample, 200))  
    for i in values:
        if i < 0:
            count +=1
            
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    
    low_95 = values.quantile(0.025)
    high_95 = values.quantile(0.975)
    
    print("Средняя прибыль: {:.2f} млн.руб".format(mean/1000000))
    print('С вероятностью 95% прибыль в данном регионе будет составлять от: {:.2f} млн.руб до: {:.2f} млн.руб'.format((low_95/1000),(high_95/1000)))
    print("95%-ый доверительный интервал: от {:.2f} млн.руб до {:.2f} млн.руб". format((confidence_interval[0]/1000),(confidence_interval[1]/1000)))
    print ('Процент вероятности убытка в данном регионе составит {:.1%}'.format(count/1000))

<div class="alert alert-block alert-success">
<b>Успех:</b> С бутстрапом тоже все в порядке, метрики считаются верным образом!  
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b>
    
Риски можно лаконично считать в 1 строку: `(values < 0).mean() * 100`
</div>                                                              


In [ ]:
bootstap(target_valid_0, prediction_0)

In [ ]:
bootstap(target_valid_1, prediction_1)

In [ ]:
bootstap(target_valid_2, prediction_2)

**Вывод** <br>
Как мы видим, самым прибыльным оказался 2 регион. Средняя прибыль в этом регионе составит 462.96 млн.рублей, при этом процент вероятности убытка в данном регионе всего 1.3%. Два других региона не подходят из-за слишком высокой вероятности убытка. Таким образом не смотря на сравнительно меньшие запасы сырья, по сравнению с другими регионами, средняя доходность со скважин во втором регионе прогнозируется выше других.

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, проведено детальное исследование и был выбран правильный регион!
</div>


## Вывод

**В ходе исследования было сделано:** <br>

1. Загрузили данные, посмотрели общую информацию о датасетах
2. Построили график распределения запасов сырья
3. Разделили данные на выборки
4. Обчили модели и сделали предсказания
5. Напечатали на экране средний запас предсказанного сырья и RMSE модели
6. Сохранили все ключевые для расчёта значения
7. Рассчитали достаточный объём сырья для безубыточной разработки новой скважины
8. Посчитали прибыль от 200 лучших скважин в каждом регионе
9. Посчитали риски и прибыль для каждого региона
10. Использовали технологию bootstrap


**Общий вывод** <br>
Средний запас ресурсов по регионам недостаточен даже для окупаемости вложений на разработку <br>
Лучшим регионом является второй, так как не смотря на сравнительно меньшие предсказания запасов сырья в 27.75 млн.баррелей, 200 лучших скважин в этом регионе дают вероятность возникновения убытка всего 1.3% и прибыль 462.96 млн.рублей.


<div class="alert alert-block alert-success">
<b>Успех:</b> Всегда приятно наблюдать подробный и структурированный итоговый вывод в конце работы!
</div>



<div class="alert alert-block alert-info">
<b>Совет: </b> Если интересно изучить тему бутстрапа глубже - я оставлю  ниже несколько интересных и полезных ссылок по теме:
        
1. "Bootstrap Hypothesis Testing in Statistics with Example" 
        
https://www.youtube.com/watch?v=9STZ7MxkNVg
        
2. "How to Calculate Bootstrap Confidence Intervals For Machine Learning Results in Python" 
        
https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/
        
3. "A Gentle Introduction to the Bootstrap Method" 

https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/
        
4. "An Introduction to the Bootstrap Method" 
        
https://towardsdatascience.com/an-introduction-to-the-bootstrap-method-58bcb51b4d60#:~:text=The%20basic%20idea%20of%20bootstrap,population%20mean)%20on%20sample%20data.&amp;text=A%20sample%20from%20population%20with%20sample%20size%20n
        
5. "Bootstrapping and Resampling in Statistics with Example" 
        
        
https://www.youtube.com/watch?v=O_Fj4q8lgmc
</div>



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован